PE파일로 부터 opcode를 추출하여 라벨링을 한 후 msgpack에 저장하는 Notebook

입력파일
* PE파일(PE, PE+)
* 각 파일에 대한 악성여부가 기재된 CSV 파일 (SHA256, 악성여부)

출력파일
* `(list, int)` 형태로 저장된 msgpack 파일

In [5]:
import glob
import pefile
from capstone import *
import parmap
import msgpack
from tqdm.auto import tqdm
import os
import csv
import shutil

In [6]:
# PE 파일에서 Opcode를 추출하는 함수
def get_opcodes(file):
    basename = os.path.splitext(os.path.basename(file))[0] # 파일 이름 추출
    file = pefile.PE(file)
    opcodes = []

    entry_point = file.OPTIONAL_HEADER.AddressOfEntryPoint # PE 파일의 진입점
    base_of_code = file.OPTIONAL_HEADER.BaseOfCode # PE 파일의 코드 섹션의 시작 주일
    code_section = file.sections[0].get_data() # PE 파일의 코드 섹션
    md = Cs(CS_ARCH_X86, CS_MODE_32 if file.FILE_HEADER.Machine == pefile.MACHINE_TYPE['IMAGE_FILE_MACHINE_I386'] else CS_MODE_64) # 32비트, 64비트 구분, 만약 32비트면 CS_MODE_32, 64비트면 CS_MODE_64

    # 진입점부터 코드 섹션의 끝까지 디스어셈블링
    for i in md.disasm(code_section, entry_point):
        opcodes.append(i.mnemonic) # Opcode만 추출
    file.close() # 메모리 절약을 위한 파일 닫기

    return (basename, opcodes) # 파일 이름과 Opcode 리스트 반환

In [7]:
def labeling(basename, opcodes, label_dict):
    label = label_dict[basename] # 파일 이름에 해당하는 레이블을 가져옴
    return (basename, (opcodes, label)) # 파일 이름, Opcode 리스트, 레이블 반환

In [8]:
def dump(file):
    global label_dict
    global dest_path
    basename, opcodes = get_opcodes(file) # 파일 이름과 Opcode 리스트를 가져옴
    basename, data = labeling(basename, opcodes, label_dict) # 파일 이름, Opcode 리스트, 레이블을 가져옴
    with open(os.path.join(dest_path,f'{basename}.msgpack'), 'wb') as f:
        msgpack.dump(data, f) # Opcode 리스트를 메시지팩으로 저장

In [9]:
# 추출할 파일 경로 입력
vir_path = input()

if vir_path == '' or os.path.isdir(vir_path) == False:
    print('Please input path of PE files')
    raise FileNotFoundError(f'{vir_path} is not found')

# 추출할 파일 경로 내의 PE 파일 목록
files = glob.glob(vir_path + '/*.vir')

In [10]:
csv_path = input()

if csv_path == '' or os.path.isfile(csv_path) == False:
    print('Please input path of CSV file for labeling')
    raise FileNotFoundError(f'{csv_path} is not exist')

# CSV 파일을 읽은 후 dict 형태로 저장 key는 sha256, value는 라벨
global label_dict
with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    label_dict = {rows[0]: rows[1] for rows in reader}

In [11]:
# 메모리 제한을 위한 청크 단위로 파일을 나누어서 라벨링
chunk_size = 1000
chunks = [files[i:i + chunk_size] for i in range(0, len(files), chunk_size)]


global dest_path
dest_path = input()

if dest_path == '' or os.path.isdir(dest_path) == False:
    print('Please input path of PE files')
    raise FileNotFoundError(f'{dest_path} is not found')

# parmap을 사용한 멀티 프로세싱
for chunk in tqdm(chunks):
    parmap.map(dump, chunk, pm_pbar=True, pm_chunksize=1)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]